In [5]:
import pandas as pd

# importing data from csv file, using the first row as column labels 'headers'
data = pd.read_csv('E:/ML/fake_news_data/news.csv/news_cleaned_2018_02_13.csv', chunksize=10000, index_col=0, lineterminator='\n')
output = pd.DataFrame({'cleaned':[], 'tokenized':[]})

In [5]:
#pip install clean-text

In [2]:
from cleantext import clean
import numpy as np

# using the clean-text library to clean the text
def clean_text(s):
    return clean(s,lower=True,                     # lowercase text
        no_urls=True,                  # replace all URLs with a special token
        no_emails=True,                # replace all email addresses with a special token
        no_numbers=True,               # replace all numbers with a special token
        replace_with_url="<URL>",
        replace_with_email="<EMAIL>",
        replace_with_number="<NUM>",
        lang="en"                   
    )


def run_clean_text(chunk):
    return chunk['content'].apply(clean_text)
    

vectorized_clean_text = np.vectorize(clean_text)

Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


In [3]:
from os import cpu_count
threads = cpu_count()
from concurrent.futures import ThreadPoolExecutor

def start_cleaning_thread(chunks):
    executor = ThreadPoolExecutor(max_workers=threads)
    futures = [executor.submit(run_clean_text, chunk) for chunk in chunks]
    executor.shutdown(wait=True)
    return(futures)

In [8]:
# cleaning the text with a custom multithreaded process
from IPython.display import clear_output
import os

chunk_size = 500
save_every = 10000
rows_number = 0
while rows_number < 30000:
    print('loading chunk')
    chunk = next(data)
    
    print('generating subchunks')
    chunks = [chunk[i:i+chunk_size] for i in range(0,chunk.shape[0],chunk_size)]
    
    print('running threads')
    futures = start_cleaning_thread(chunks)
    

    print('concatenating results')
    result = pd.DataFrame()
    for future in futures:
        #print((future.result().to_frame()['content']))
        #wait()
        df = pd.DataFrame()
        df['cleaned'] = future.result()
        result = pd.concat([result,df], ignore_index=True)

        
    #output['cleaned'] = pd.concat([output,result['cleaned']], ignore_index=True)
    result = result.reset_index(drop=True)
    output['cleaned'] = output['cleaned'].append(result['cleaned'])    
    
    rows_number += len(chunk)
    clear_output()
    print('finished', rows_number, 'rows')

    # save the output every save_every rows
    if rows_number % save_every == 0:
        output.to_csv('output.csv', index=False, mode='a', header= (not os.path.exists('output.csv')))
        output = pd.DataFrame()  # clear the output to free up memory

loading chunk
generating subchunks
running threads
concatenating results


C:\Users\musta\AppData\Local\Temp\ipykernel_16844\3768539914.py:31: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output['cleaned'] = output['cleaned'].append(result['cleaned'])
C:\Users\musta\AppData\Local\Temp\ipykernel_16844\3768539914.py:31: FutureWarning: reindexing with a non-unique Index is deprecated and will raise in a future version.
  output['cleaned'] = output['cleaned'].append(result['cleaned'])


ValueError: cannot reindex on an axis with duplicate labels

In [24]:
# using swifter did not yield any speedup, rather it is a bit slower
import swifter
data_temp = next(data)
result = data_temp['content'].swifter.apply(clean_text)
output = pd.concat([output,result.to_frame()], ignore_index=True)

Pandas Apply:   0%|          | 0/10000 [00:00<?, ?it/s]

In [28]:
# using pandas apply is also slighty slower
data_temp = next(data)
content = data_temp['content'].values
content = vectorized_clean_text(content)
output = pd.concat([output,result.to_frame()], ignore_index=True)

In [16]:
output

,cleaned,tokenized,content
0,NaN,NaN,"life is an illusion, at least on a quantum lev..."
1,NaN,NaN,"unfortunately, he hasn't yet attacked her for ..."
2,NaN,NaN,the los angeles police department has been den...
3,NaN,NaN,the white house has decided to quietly withdra...
4,NaN,NaN,"""the time has come to cut off the tongues of t..."
...,...,...,...
995,NaN,NaN,"in a shocking, but not unprecedented christmas..."
996,NaN,NaN,thrilling thursday nasdaq <num> is our next su...
997,NaN,NaN,obama facing environmental lawsuit from chicag...
998,NaN,NaN,fake news is satanic says pope\n% of readers t...


In [27]:
import nltk
from nltk.tokenize import  word_tokenize # this tokenizer is arbitrary, but we can use it for now and analyze the results later
#nltk.download('punkt')

data['processed_content'] = data['cleaned_content'].apply(lambda t : (word_tokenize(t)))
word_frq_pre_stopwords_removal = nltk.FreqDist(data['processed_content'].sum())

In [28]:
from nltk.corpus import stopwords
# the following line must be uncommented first time this package is used in the environment
#nltk.download('stopwords')

def remove_stopwords(s):
    # the stopwords from the library are put in a set for faster lookup
    words_to_remove = set(stopwords.words('english')) # this line makes the code ~400x faster!
    # checking each indiviudal token to see if its in the set of stopwords
    return [w for w in s if w not in words_to_remove]
# the stepwordless text is stored in a new column 'processed_content' overriding the previous data
data['processed_content'] = data['processed_content'].apply(remove_stopwords)
word_frq_post_stopwords_removal = nltk.FreqDist(data['processed_content'].sum())

In [29]:
print('vocab size before removing stopwords', len(word_frq_pre_stopwords_removal))
print('token number before removing stopwords', sum(word_frq_pre_stopwords_removal.values()))
print('vocab size after removing stopwords', len(word_frq_post_stopwords_removal))
print('token number after removing stopwords', sum(word_frq_post_stopwords_removal.values()))
print('vocab size reduction', (len(word_frq_pre_stopwords_removal) - len(word_frq_post_stopwords_removal)) / len(word_frq_pre_stopwords_removal) * 100, '%')
print('token number reduction', (sum(word_frq_pre_stopwords_removal.values()) - sum(word_frq_post_stopwords_removal.values())) / sum(word_frq_pre_stopwords_removal.values()) * 100, '%')


vocab size before removing stopwords 16586
token number before removing stopwords 200995
vocab size after removing stopwords 16454
token number after removing stopwords 126763
vocab size reduction 0.7958519233088147 %
token number reduction 36.93226199656708 %


In [30]:
from nltk.stem import PorterStemmer # this stemmer is arbitrary, but we can use it for now and analyze the results later
# the following line must be uncommented first time this package is used in the environment
#nltk.download('wordnet')

# the stemmer reduces the tokens (words) to their root form
def stem_words(l):
    ps = PorterStemmer()
    return [ps.stem(w) for w in l]
# the stemmed text is stored in the column 'processed_content' overriding the previous data
data['processed_content'] = data['processed_content'].apply(stem_words)

In [31]:
word_frq_post_stemming = nltk.FreqDist(data['processed_content'].sum())
print('vocab reduction after stemming', (len(word_frq_post_stopwords_removal) - len(word_frq_post_stemming)) / len(word_frq_post_stopwords_removal) * 100, '%')

vocab reduction after stemming 31.445241278716423 %


In [33]:
data.to_csv('news_sample_processed.csv')

In [35]:
data

,id,domain,type,url,content,scraped_at,inserted_at,updated_at,title,authors,keywords,meta_keywords,meta_description,tags,summary,cleaned_content,processed_content
0,141,awm.com,unreliable,http://awm.com/church-congregation-brings-gift...,Sometimes the power of Christmas will make you...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Church Congregation Brings Gift to Waitresses ...,Ruth Harris,NaN,[''],NaN,NaN,NaN,sometimes the power of christmas will make you...,"[sometim, power, christma, make, wild, wonder,..."
1,256,beforeitsnews.com,fake,http://beforeitsnews.com/awakening-start-here/...,AWAKENING OF 12 STRANDS of DNA – “Reconnecting...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,AWAKENING OF 12 STRANDS of DNA – “Reconnecting...,Zurich Times,NaN,[''],NaN,NaN,NaN,"awakening of <num> strands of dna ""reconnectin...","[awaken, <, num, >, strand, dna, ``, reconnect..."
2,700,cnnnext.com,unreliable,http://www.cnnnext.com/video/18526/never-hike-...,Never Hike Alone: A Friday the 13th Fan Film U...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Never Hike Alone - A Friday the 13th Fan Film ...,NaN,NaN,[''],Never Hike Alone: A Friday the 13th Fan Film ...,NaN,NaN,never hike alone: a friday the 13th fan film u...,"[never, hike, alon, :, friday, 13th, fan, film..."
3,768,awm.com,unreliable,http://awm.com/elusive-alien-of-the-sea-caught...,"When a rare shark was caught, scientists were ...",2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Elusive ‘Alien Of The Sea ‘ Caught By Scientis...,Alexander Smith,NaN,[''],NaN,NaN,NaN,"when a rare shark was caught, scientists were ...","[rare, shark, caught, ,, scientist, left, blun..."
4,791,bipartisanreport.com,clickbait,http://bipartisanreport.com/2018/01/21/trumps-...,Donald Trump has the unnerving ability to abil...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Trump’s Genius Poll Is Complete & The Results ...,Gloria Christie,NaN,[''],NaN,NaN,NaN,donald trump has the unnerving ability to abil...,"[donald, trump, unnerv, abil, abil, creat, rea..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,39259,beforeitsnews.com,fake,http://beforeitsnews.com/economy/2017/12/priso...,"Prison for Rahm, God’s Work And Many Others\r\...",2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,"Prison for Rahm, God’s Work And Many Others",NaN,NaN,[''],NaN,NaN,NaN,"prison for rahm, god's work and many others\nh...","[prison, rahm, ,, god, 's, work, mani, other, ..."
246,39468,beforeitsnews.com,fake,http://beforeitsnews.com/diy/2017/11/4-useful-...,4 Useful Items for Your Tiny Home\r\n\r\nHeadl...,2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,4 Useful Items for Your Tiny Home,Dimitry K,NaN,[''],NaN,NaN,NaN,<num> useful items for your tiny home\nheadlin...,"[<, num, >, use, item, tini, home, headlin, :,..."
247,39477,www.newsmax.com,NaN,https://www.newsmax.com/politics/michael-hayde...,Former CIA Director Michael Hayden said Thursd...,2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Michael Hayden: We Should Be 'Frightened' by T...,Todd Beamon,NaN,"['michael hayden', 'sthole countries', 'daca',...",President Donald Trump's reported remarks abou...,"Homeland Security, Trump Administration, Immig...",NaN,former cia director michael hayden said thursd...,"[former, cia, director, michael, hayden, said,..."
248,39550,www.newsmax.com,NaN,https://www.newsmax.com/newsfront/antonio-saba...,Antonio Sabato Jr. says Hollywood's liberal el...,2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Antonio Sabato Jr.: It's Oprah or Bust for Hol...,Bill Hoffmann,NaN,"['antonio sabato jr', 'oprah winfrey', 'presid...",Antonio Sabato Jr. says Hollywood's liberal el...,"Trump Administration, ISIS/Islamic State, News